In [ ]:
from docksuitex import Protein, Ligand, PocketFinder, VinaDocking, AD4Docking
from docksuitex.utils import clean_temp_folder, fetch_pdb, fetch_sdf, view_molecule, parse_vina_log_to_csv, parse_ad4_dlg_to_csv
clean_temp_folder()
from pathlib import Path

In [ ]:
# Fetch and prepare protein
Protein_path = fetch_pdb(pdbid = "1HVR")
prot = Protein(Protein_path)
prot.prepare()

In [ ]:
# Find pockets using p2rank
pf = PocketFinder(prot.pdb_path)
pockets = pf.run()
pockets

In [ ]:
# Fetch ligands
fetch_sdf(cid = "338", save_dir="ligands")
fetch_sdf(cid = "5288826", save_dir="ligands")

# Set up ligand folder
ligand_folder = Path("ligands")  # Your folder with SDF files
ligand_files = sorted(ligand_folder.glob("*.sdf"))

### Docks each ligand against a single receptor (at the first pocket).

In [ ]:
# Loop through each ligand
for ligand_file in ligand_files:
    print(f"\nProcessing: {ligand_file.name}")
    
    # Prepare ligand
    lig = Ligand(str(ligand_file))
    lig.prepare(minimize=None)
    
    # Perform docking with first pocket
    print(f"Docking receptor: {prot.pdbqt_path.name}, ligand: {lig.pdbqt_path.name}")
    
    # Using AutoDock4
    ad4 = AD4Docking(
        receptor=prot.pdbqt_path,
        ligand=lig.pdbqt_path,
        grid_center=pockets[0]['center'],

        #Default Parameters
        grid_size=(60,60,60),
        spacing=0.375,     
        dielectric=-0.1465, 
        smooth=0.5,          
        ga_pop_size=150,
        ga_num_evals=2500000,
        ga_num_generations=27000,
        ga_elitism=1,
        ga_mutation_rate=0.02,
        ga_crossover_rate=0.8,
        ga_run=10,
        rmstol=2.0,
    )

    ad4.run()
    ad4.save_results(f"ad4_results/{prot.pdbqt_path.stem}_{lig.pdbqt_path.stem}_pocket1_docking")

# Parse AD4 results
parse_ad4_dlg_to_csv("ad4_results", "ad4_results/ad4_summary.csv")


### Dock each ligand against a single receptor at all the predicted protein pockets


In [ ]:
# Loop through each ligand
for ligand_file in ligand_files:
    print(f"\nProcessing: {ligand_file.name}")
    
    # Prepare ligand
    lig = Ligand(str(ligand_file))
    lig.prepare(minimize=None)

    # Loop through each pocket
    for pocket in pockets: #pockets[:3], For first three pockets
        center = pocket["center"]
        rank = pocket["rank"]

        print(f"Docking receptor: {prot.pdbqt_path.name}, ligand: {lig.pdbqt_path.name}, pocket: {rank}")

        # Using Vina
        vina = VinaDocking(
            receptor=prot.pdbqt_path,
            ligand=lig.pdbqt_path,
            grid_center=center,
            grid_size=(40, 40, 40),
            exhaustiveness=8,
            num_modes=9
        )
        
        vina.run()
        vina.save_results(f"vina_results/{prot.pdbqt_path.stem}_{lig.pdbqt_path.stem}_pocket_{rank}_docking")

# Parse vina results
parse_vina_log_to_csv("vina_results", "vina_results/vina_summary.csv")

